In [12]:
import torch
import numpy as np

import _game
import _bot_b2

import importlib
importlib.reload(_game)
importlib.reload(_bot_b2)

from _game import Game, Game_Hokom, Game_Sun, Bot_Random


In [17]:
models_b2 = [_bot_b2.Model(i) for i in range(4)]
cp_b2 = torch.load("model_b2/cp111000.pt")
[models_b2[i].load_state_dict(cp_b2["models_state_dict"][i]) for i in range(4)]
bots_b2 = [_bot_b2.Bot(models_b2) for i in range(4)]

In [16]:
game = Game_Hokom(verbose=1)
game.register_bots(bots=bots_b2)
game.whole_game()

hokum: ♠
Bot 0 played ♣8 (♠7 ♠A ♥8 ♥Q ♥K ♣Q ♦A)
Bot 1 played ♣9 (♠9 ♥7 ♥9 ♥A ♣A ♦8 ♦J)
Bot 2 played ♣7 (♠8 ♠Q ♥10 ♣J ♣K ♦7 ♦K)
Bot 3 played ♣10 (♠10 ♠J ♠K ♥J ♦9 ♦10 ♦Q)
Round:1 score:-10 Scores:-10
Bot 3 played ♥J (♠10 ♠J ♠K ♦9 ♦10 ♦Q)
Bot 0 played ♥K (♠7 ♠A ♥8 ♥Q ♣Q ♦A)
Bot 1 played ♥A (♠9 ♥7 ♥9 ♣A ♦8 ♦J)
Bot 2 played ♥10 (♠8 ♠Q ♣J ♣K ♦7 ♦K)
Round:2 score:-27 Scores:-37
Bot 1 played ♠9 (♥7 ♥9 ♣A ♦8 ♦J)
Bot 2 played ♠Q (♠8 ♣J ♣K ♦7 ♦K)
Bot 3 played ♠10 (♠J ♠K ♦9 ♦10 ♦Q)
Bot 0 played ♠A (♠7 ♥8 ♥Q ♣Q ♦A)
Round:3 score:-38 Scores:-75
Bot 1 played ♥7 (♥9 ♣A ♦8 ♦J)
Bot 2 played ♠8 (♣J ♣K ♦7 ♦K)
Bot 3 played ♠K (♠J ♦9 ♦10 ♦Q)
Bot 0 played ♥8 (♠7 ♥Q ♣Q ♦A)
Round:4 score:-4 Scores:-79
Bot 3 played ♦Q (♠J ♦9 ♦10)
Bot 0 played ♦A (♠7 ♥Q ♣Q)
Bot 1 played ♦8 (♥9 ♣A ♦J)
Bot 2 played ♦7 (♣J ♣K ♦K)
Round:5 score:14 Scores:-65
Bot 0 played ♥Q (♠7 ♣Q)
Bot 1 played ♥9 (♣A ♦J)
Bot 2 played ♣K (♣J ♦K)
Bot 3 played ♠J (♦9 ♦10)
Round:6 score:-27 Scores:-92
Bot 3 played ♦9 (♦10)
Bot 0 played ♠7 (♣Q)
Bot 1 pl

In [18]:
game = Game_Hokom(verbose=1)
game.register_bots(bots=[_bot_b2.Bot(models_b2), Bot_Random(), _bot_b2.Bot(models_b2), Bot_Random()])
game.whole_game()

hokum: ♠
Bot 3 played ♠K (♠7 ♠J ♥J ♥K ♣7 ♣K ♣A)
Bot 0 played ♠8 (♠A ♥A ♣J ♣Q ♦7 ♦8 ♦J)
Bot 1 played ♠9 (♠10 ♥7 ♥10 ♣8 ♣9 ♦9 ♦Q)
Bot 2 played ♠Q (♥8 ♥9 ♥Q ♣10 ♦10 ♦K ♦A)
Round:1 score:-21 Scores:-21
Bot 1 played ♣9 (♠10 ♥7 ♥10 ♣8 ♦9 ♦Q)
Bot 2 played ♣10 (♥8 ♥9 ♥Q ♦10 ♦K ♦A)
Bot 3 played ♣A (♠7 ♠J ♥J ♥K ♣7 ♣K)
Bot 0 played ♣Q (♠A ♥A ♣J ♦7 ♦8 ♦J)
Round:2 score:-24 Scores:-45
Bot 3 played ♣7 (♠7 ♠J ♥J ♥K ♣K)
Bot 0 played ♣J (♠A ♥A ♦7 ♦8 ♦J)
Bot 1 played ♣8 (♠10 ♥7 ♥10 ♦9 ♦Q)
Bot 2 played ♥Q (♥8 ♥9 ♦10 ♦K ♦A)
Round:3 score:5 Scores:-40
Bot 0 played ♦J (♠A ♥A ♦7 ♦8)
Bot 1 played ♦Q (♠10 ♥7 ♥10 ♦9)
Bot 2 played ♦A (♥8 ♥9 ♦10 ♦K)
Bot 3 played ♠7 (♠J ♥J ♥K ♣K)
Round:4 score:-16 Scores:-56
Bot 3 played ♣K (♠J ♥J ♥K)
Bot 0 played ♠A (♥A ♦7 ♦8)
Bot 1 played ♠10 (♥7 ♥10 ♦9)
Bot 2 played ♥9 (♥8 ♦10 ♦K)
Round:5 score:25 Scores:-31
Bot 0 played ♦8 (♥A ♦7)
Bot 1 played ♦9 (♥7 ♥10)
Bot 2 played ♦10 (♥8 ♦K)
Bot 3 played ♠J (♥J ♥K)
Round:6 score:-30 Scores:-61
Bot 3 played ♥K (♥J)
Bot 0 played ♥A (♦7)
Bot